In [18]:
import torch
from torchvision import transforms, models
import cv2
from PIL import Image
import os
from pathlib import Path
import shutil


In [19]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_orientation_model(checkpoint_path):
    """
    Loads the trained orientation model from the given checkpoint path.
    """
    model = models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, 2)  # Binary classification: left (0) vs. right (1)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()
    print(f"Model loaded from {checkpoint_path}")
    return model

# Example: Load the model
checkpoint_path = "../checkpoints/orientation_classifier_checkpoint.pth"
model = load_orientation_model(checkpoint_path)


Model loaded from ../checkpoints/orientation_classifier_checkpoint.pth


/var/folders/w7/n9y0bc4n5zgb8gg118j_j66h0000gn/T/ipykernel_74321/3367737889.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map

In [20]:
def predict_orientation(model, image_path):
    """
    Predicts the orientation of the car in the image.
    Returns 'left' if the car faces left, 'right' if it faces right.
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        return "left" if predicted.item() == 0 else "right"


In [21]:
def flip_labels_x(label_path, output_label_dir):
    """
    Flips the x-coordinates of labels and saves them to a new directory.

    Parameters:
        label_path (str): Path to the original label file.
        output_label_dir (str): Directory to save the flipped labels.
    """
    os.makedirs(output_label_dir, exist_ok=True)

    flipped_labels = []
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x_center = 1 - x_center  # Flip the x-coordinate
            flipped_labels.append(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    output_label_path = os.path.join(output_label_dir, os.path.basename(label_path))
    with open(output_label_path, 'w') as f:
        f.write("\n".join(flipped_labels))
    
    print(f"Flipped labels saved to {output_label_path}")


In [22]:
def flip_image(image_path, output_image_dir):
    """
    Flips an image horizontally and saves it to the output directory.

    Parameters:
        image_path (str): Path to the original image.
        output_image_dir (str): Directory to save the flipped image.
    """
    os.makedirs(output_image_dir, exist_ok=True)

    image = cv2.imread(str(image_path))
    flipped_image = cv2.flip(image, 1)
    output_image_path = os.path.join(output_image_dir, os.path.basename(image_path))
    cv2.imwrite(output_image_path, flipped_image)
    
    print(f"Flipped image saved to {output_image_path}")
    return output_image_path


In [23]:
def process_images_and_labels(model, images_dir, labels_dir, output_image_dir, output_label_dir):
    """
    Processes all images: identifies flipped images, flips them, and flips the labels' x-coordinates.

    Parameters:
        model (torch.nn.Module): Trained orientation model.
        images_dir (str): Directory containing the input images.
        labels_dir (str): Directory containing the label files.
        output_image_dir (str): Directory to save processed images.
        output_label_dir (str): Directory to save flipped labels.
    """
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    for image_path in Path(images_dir).glob("*.jpg"):
        label_path = Path(labels_dir) / f"{image_path.stem}.txt"
        
        if label_path.exists():
            # Predict orientation
            orientation = predict_orientation(model, str(image_path))
            flipped = False

            if orientation == "right":
                flipped = True
                # Flip the image and save
                flip_image(str(image_path), output_image_dir)
                # Flip the labels and save
                flip_labels_x(str(label_path), output_label_dir)
            else:
                # Copy the original image and label to the output directories
                shutil.copy(str(image_path), os.path.join(output_image_dir, image_path.name))
                shutil.copy(str(label_path), os.path.join(output_label_dir, label_path.name))
                print(f"Image and labels copied to {output_image_dir} and {output_label_dir}")
        else:
            print(f"Label file not found for {image_path}")


In [ ]:
# Example usage
checkpoint_path = "../checkpoints/orientation_classifier_checkpoint.pth"
images_dir = "../data/processed/images_cropped"
labels_dir = "../data/processed/labels"
output_image_dir = "../data/output/wheel_images_flipped"
output_label_dir = "../data/output/wheel_labels_flipped"

model = load_orientation_model(checkpoint_path)

process_images_and_labels(model, images_dir, labels_dir, output_image_dir, output_label_dir)


/var/folders/w7/n9y0bc4n5zgb8gg118j_j66h0000gn/T/ipykernel_74321/3367737889.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map

Model loaded from ../checkpoints/orientation_classifier_checkpoint.pth
Image and labels copied to ../data/output/wheel_images_flipped and ../data/output/wheel_labels_flipped
Image and labels copied to ../data/output/wheel_images_flipped and ../data/output/wheel_labels_flipped
Image and labels copied to ../data/output/wheel_images_flipped and ../data/output/wheel_labels_flipped
Flipped image saved to ../data/output/wheel_images_flipped/M_For_09.jpg
Flipped labels saved to ../data/output/wheel_labels_flipped/M_For_09.txt
Image and labels copied to ../data/output/wheel_images_flipped and ../data/output/wheel_labels_flipped
Image and labels copied to ../data/output/wheel_images_flipped and ../data/output/wheel_labels_flipped
Flipped image saved to ../data/output/wheel_images_flipped/S_Aud_07.jpg
Flipped labels saved to ../data/output/wheel_labels_flipped/S_Aud_07.txt
Flipped image saved to ../data/output/wheel_images_flipped/C_Lex_05.jpg
Flipped labels saved to ../data/output/wheel_labels_

## Retrieve fron wheel

In [ ]:
def get_front_wheel(label_path, wheel_class=1):
    """
    Extracts the coordinates of the front wheel (class 1) from a label file.

    Parameters:
        label_path (str): Path to the label file.
        wheel_class (int): The class ID for wheels (default: 1).

    Returns:
        tuple: Coordinates of the front wheel in the format (x_center, y_center, width, height),
               or None if no wheels are found.
    """
    wheels = []

    # Read the label file and collect all wheels
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            if int(class_id) == wheel_class:
                wheels.append((x_center, y_center, width, height))

    if not wheels:
        print(f"No wheels found in {label_path}")
        return None

    # Identify the front wheel (wheel with the smallest x_center)
    front_wheel = min(wheels, key=lambda wheel: wheel[0])
    
    print(f"Front wheel found in {label_path}: {front_wheel}")
    return front_wheel

In [ ]:


# Get the front wheel coordinates
output_label_dir = "../data/output/wheel_labels_flipped/A_Ast_01.txt"
front_wheel = get_front_wheel(output_label_dir)

if front_wheel:
    print(f"Front wheel coordinates: {front_wheel}")


Front wheel found in ../data/output/wheel_labels_flipped/A_Ast_01.txt: (0.209473, 0.724609, 0.135742, 0.134766)
Front wheel coordinates: (0.209473, 0.724609, 0.135742, 0.134766)


# Until here

In [29]:
import cv2
import os
from pathlib import Path
from collections import defaultdict


In [30]:
def get_front_wheel_dimensions(label_path, wheel_class=1):
    """
    Extracts the width and height of the front wheel (class 1) from a label file.

    Parameters:
        label_path (str): Path to the label file in YOLO format where class_id = 1 => rim.
        wheel_class (int): The class ID for wheels (default: 1).

    Returns:
        tuple: Width and height of the front wheel, or None if no wheels are found.
    """
    smallest_x = float('inf')  
    front_wheel = None        

    with open(label_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            if int(class_id) == wheel_class:
                if x_center < smallest_x:
                    smallest_x = x_center
                    front_wheel = (width, height)

    if front_wheel:
        return front_wheel
    else:
        return None


In [31]:
def resize_image_by_reference_wheel(image_path, label_path, output_dir, reference_width, reference_height, resolution=1024):
    """
    Resizes the image so the front wheel matches the reference width and height.

    Parameters:
        image_path (str): Path to the input image.
        label_path (str): Path to the corresponding label file.
        output_dir (str): Directory to save the resized image.
        reference_width (float): Width of the reference front wheel (normalized).
        reference_height (float): Height of the reference front wheel (normalized).
        resolution (int): Resolution of the image (default: 1024).
    """
    os.makedirs(output_dir, exist_ok=True)

    # Get the current front wheel dimensions
    current_wheel = get_front_wheel_dimensions(label_path)
    if not current_wheel:
        print(f"No front wheel found in {label_path}. Skipping.")
        return

    current_width, current_height = current_wheel

    # Calculate the scaling factor
    scale_factor_width = reference_width / current_width
    scale_factor_height = reference_height / current_height
    scale_factor = (scale_factor_width + scale_factor_height) / 2  # Average scaling factor

    # Read the image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Could not read image at {image_path}.")
        return

    # Resize the image
    new_width = int(image.shape[1] * scale_factor)
    new_height = int(image.shape[0] * scale_factor)
    resized_image = cv2.resize(image, (new_width, new_height))

    # Save the resized image
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, resized_image)
    print(f"Resized image saved to {output_path}")

    # Update the labels to reflect the new image dimensions
    update_labels_for_resized_image(label_path, output_dir, scale_factor)


In [36]:
def update_labels_for_resized_image(label_path, output_dir, scale_factor):
    """
    Updates the label coordinates to match the resized image and saves them to the output directory.

    Parameters:
        label_path (str): Path to the original label file.
        output_dir (str): Directory to save the updated label file.
        scale_factor (float): The scaling factor applied to the image.
    """
    os.makedirs(output_dir, exist_ok=True)
    output_label_path = os.path.join(output_dir, os.path.basename(label_path))

    updated_labels = []
    with open(label_path, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            # Update coordinates based on the scale factor
            x_center *= scale_factor
            y_center *= scale_factor
            width *= scale_factor
            height *= scale_factor
            updated_labels.append(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Save the updated labels
    with open(output_label_path, 'w') as f:
        f.write("\n".join(updated_labels))
    
    print(f"Updated labels saved to {output_label_path}")


In [37]:
def process_images_by_category(images_dir, labels_dir, output_dir, reference_files):
    """
    Processes all images in each category using a reference car's front wheel dimensions.

    Parameters:
        images_dir (str): Directory containing the input images.
        labels_dir (str): Directory containing the label files.
        output_dir (str): Directory to save the resized images and updated labels.
        reference_files (dict): Dictionary where keys are categories and values are (image_path, label_path) tuples for the reference car.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Group images by category
    images_by_category = defaultdict(list)
    for image_path in Path(images_dir).glob("*.jpg"):
        category = image_path.stem.split("_")[0]
        images_by_category[category].append(image_path)

    # Process each category
    for category, image_paths in images_by_category.items():
        if category not in reference_files:
            print(f"No reference file provided for category {category}. Skipping.")
            continue

        ref_image_path, ref_label_path = reference_files[category]
        reference_wheel = get_front_wheel_dimensions(ref_label_path)

        if not reference_wheel:
            print(f"No front wheel found in reference file for category {category}. Skipping.")
            continue

        ref_width, ref_height = reference_wheel

        for image_path in image_paths:
            label_path = Path(labels_dir) / f"{image_path.stem}.txt"
            if label_path.exists():
                resize_image_by_reference_wheel(str(image_path), str(label_path), output_dir, ref_width, ref_height)
            else:
                print(f"Label file not found for {image_path}")


In [41]:
images_dir = "../data/with_labels/images_flipped"
labels_dir = "../data/with_labels/json_flipped"
output_dir = "../data/output/resized_images"

reference_files = {
    "A": ("../data/output/wheel_images_flipped/A_Ast_01.jpg", "../data/output/wheel_labels_flipped/A_Ast_01.txt"),
    #"B": ("../data/processed_images/B_Bmw_01.jpg", "../data/processed_labels/B_Bmw_01.txt"),
}

process_images_by_category(images_dir, labels_dir, output_dir, reference_files)


No reference file provided for category D. Skipping.
No reference file provided for category C. Skipping.
No reference file provided for category M. Skipping.
No reference file provided for category E. Skipping.
No reference file provided for category S. Skipping.
No reference file provided for category J. Skipping.
Label file not found for ../data/with_labels/images_flipped/A_Hyu_06.jpg
Label file not found for ../data/with_labels/images_flipped/A_Hyu_12.jpg
Label file not found for ../data/with_labels/images_flipped/A_Ope_05.jpg
Label file not found for ../data/with_labels/images_flipped/A_Fia_03.jpg
Label file not found for ../data/with_labels/images_flipped/A_Fia_02.jpg
Label file not found for ../data/with_labels/images_flipped/A_Ope_04.jpg
Label file not found for ../data/with_labels/images_flipped/A_Hyu_07.jpg
Label file not found for ../data/with_labels/images_flipped/A_Hyu_11.jpg
Label file not found for ../data/with_labels/images_flipped/A_Ope_06.jpg
Label file not found for 